<a href="https://colab.research.google.com/github/AkhmadRamadani/Tugas-7-BigData/blob/main/rating_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install PySpark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=3fef3709698fd073c6fb418303cc58e90d9b26885c44cfff183821c15d397b29
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from pyspark.sql import SparkSession
from pyspark.mllib.clustering import KMeans, KMeansModel
from numpy import array
from math import sqrt

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col

In [5]:
spark = SparkSession.builder.appName("Movie Lens").getOrCreate()
ratings = spark.read.csv("/content/drive/MyDrive/Semester 6/BIG Data/Week 15/ml-latest-small/ratings.csv", header=True, inferSchema=True)
# Check if everything is ok
ratings.show(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
+------+-------+------+---------+
only showing top 5 rows



In [6]:
class Rating:
    def __init__(self, userId, movieId, rating, timestamp):
        self.userId = int(userId)
        self.movieId = int(movieId)
        self.rating = float(rating)
        self.timestamp = float(timestamp)

def parseRating(str):
    fields = str.split(",")
    assert(len(fields) == 4)
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]), float(fields[3]))

In [7]:
# Split data into 80% train 20% test
training, test = ratings.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")

model = als.fit(training)
model.write().overwrite().save("rama_model")

# Prepare the recommendations
predictions = model.transform(test)
squaredErrors = predictions.withColumn("squaredError", (col("rating") - col("prediction")) ** 2)
mse = predictions.withColumn("diff", col("rating") - col("prediction")).select((col("diff") ** 2).alias("squared_diff")).filter(~col("squared_diff").isNull()).agg({"squared_diff": "sum"}).collect()[0][0]

predictions.show(10)

# Menyimpan Hasil Prediksi
predictions.write.format("csv").save("ml-predictions.csv")


+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|     1|    296|   3.0|964982967|  4.802364|
|     1|    333|   5.0|964981179| 4.6759872|
|     1|      6|   4.0|964982224| 3.9875753|
|     1|    919|   5.0|964982475| 4.1383686|
|     1|    163|   5.0|964983650| 3.5266614|
|     1|    553|   5.0|964984153| 4.6101193|
|     1|     50|   5.0|964982931|  4.992085|
|     1|    151|   5.0|964984041| 2.9001954|
|     1|    592|   4.0|964982271| 4.2358203|
|     1|    110|   4.0|964982176|  5.115114|
+------+-------+------+---------+----------+
only showing top 10 rows

